In [1]:
!git clone https://github.com/elayemu/10-Academy-W2-User-Insight-Analysis.git

Cloning into '10-Academy-W2-User-Insight-Analysis'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), done.


In [2]:
%cd 10-Academy-W2-User-Insight-Analysis

/kaggle/working/10-Academy-W2-User-Insight-Analysis


In [3]:
!git pull origin main

From https://github.com/elayemu/10-Academy-W2-User-Insight-Analysis
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.4 MB/s eta 0:00:00


# Import Required Libraries

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import streamlit as st

# Load the Dataset

In [6]:
# Load the dataset and description of column names
data_path = '/kaggle/input/w2-data/Week2_challenge_data_source.xlsx'
description_path = '/kaggle/input/w2-data/Field Descriptions.xlsx'

# Read the Excel files
data = pd.read_excel(data_path)
description = pd.read_excel(description_path)

# Display the first few rows of the dataset
data.head(), description.head()

(              Bearer Id               Start  Start ms                 End  \
 0  13114483460844900352 2019-04-04 12:01:18     770.0 2019-04-25 14:35:31   
 1  13114483482878900224 2019-04-09 13:04:04     235.0 2019-04-25 08:15:48   
 2  13114483484080500736 2019-04-09 17:42:11       1.0 2019-04-25 11:58:13   
 3  13114483485442799616 2019-04-10 00:31:25     486.0 2019-04-25 07:36:35   
 4  13114483499480700928 2019-04-12 20:10:23     565.0 2019-04-25 10:40:32   
 
    End ms  Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
 0   662.0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
 1   606.0  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
 2   652.0  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
 3   171.0  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
 4   954.0  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   
 
   Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
 0   9164566995485190  ...          15854611.0    

# Data Preparation and Cleaning

In [7]:
# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values[missing_values > 0])

# Select only numeric columns and calculate the mean
numeric_cols = data.select_dtypes(include=['number']).columns

# Fill missing values with the mean of numeric columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

print("Missing values filled. Current missing values status:\n", data.isnull().sum())

Missing values in each column:
 Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)                             754

# User Overview Analysis

In [8]:
# Top 10 handsets
top_handsets = data['Handset Type'].value_counts().head(10)
print(top_handsets)

Handset Type
Huawei B528S-23A                19752
Apple iPhone 6S (A1688)          9419
Apple iPhone 6 (A1586)           9023
undefined                        8987
Apple iPhone 7 (A1778)           6326
Apple iPhone Se (A1723)          5187
Apple iPhone 8 (A1905)           4993
Apple iPhone Xr (A2105)          4568
Samsung Galaxy S8 (Sm-G950F)     4520
Apple iPhone X (A1901)           3813
Name: count, dtype: int64


In [9]:
# Top 3 handset manufacturers
top_manufacturers = data['Handset Manufacturer'].value_counts().head(3)
print(top_manufacturers)

Handset Manufacturer
Apple      59565
Samsung    40839
Huawei     34423
Name: count, dtype: int64


In [10]:
# Top 5 handsets per top 3 manufacturers
top_handsets_per_manufacturer = {}
for manufacturer in top_manufacturers.index:
    top_handsets_per_manufacturer[manufacturer] = data[data['Handset Manufacturer'] == manufacturer]['Handset Type'].value_counts().head(5)
    

In [11]:
print(top_handsets_per_manufacturer)

{'Apple': Handset Type
Apple iPhone 6S (A1688)    9419
Apple iPhone 6 (A1586)     9023
Apple iPhone 7 (A1778)     6326
Apple iPhone Se (A1723)    5187
Apple iPhone 8 (A1905)     4993
Name: count, dtype: int64, 'Samsung': Handset Type
Samsung Galaxy S8 (Sm-G950F)    4520
Samsung Galaxy A5 Sm-A520F      3724
Samsung Galaxy J5 (Sm-J530)     3696
Samsung Galaxy J3 (Sm-J330)     3484
Samsung Galaxy S7 (Sm-G930X)    3199
Name: count, dtype: int64, 'Huawei': Handset Type
Huawei B528S-23A                  19752
Huawei E5180                       2079
Huawei P20 Lite Huawei Nova 3E     2021
Huawei P20                         1480
Huawei Y6 2018                      997
Name: count, dtype: int64}
